In [2]:
%load_ext autoreload
%autoreload 2

In [118]:
import openai
import random
import re

import util

In [10]:
f = open('/Users/ericmelz/Documents/code/openai-quickstart-python-emelz/.env', 'r')

KEY='OPENAI_API_KEY'
l=f.readline()
api_key = 'undefined!'
while l:
    if l.startswith(KEY):
        print('FOUND')
        api_key = l[len(KEY)+1:-1]
        break
    l = f.readline()

f.close()

openai.api_key=api_key

FOUND


In [4]:
query = 'Ray buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?'

In [148]:
noun_preamble = 'Given the following question, list the nouns.  Return the answer as a list of bullet points, or say None if there are no items.'
pronoun_preamble = 'Given the following question, list the pronouns.  Return the answer as a list of bullet points, or say None if there are no items.'
proper_noun_preamble = 'Given the following question, list the proper nouns.  Return the answer as a list of bullet points, or say None if there are no items.'
male_name_preamble = 'Given the following question, list the males mentioned in the question.  Return the answer as a list of bullet points, or say None if there are no items.'
female_name_preamble = 'Given the following question, list the females mentioned in the question.  Return the answer as a list of bullet points, or say None if there are no items'

In [150]:
noun_query = f'{noun_preamble}\n{query}'
pronoun_query = f'{pronoun_preamble}\n{query}'
proper_noun_query = f'{proper_noun_preamble}\n{query}'
male_query = f'{male_name_preamble}\n{query}'
female_query = f'{female_name_preamble}\n{query}'

In [55]:
print(noun_query)

Given the following question, list the nouns.  Return the answer as a list of bullet points, or say None if there are no items.
Ray buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?


In [56]:
print(pronoun_query)

Given the following question, list the pronouns.  Return the answer as a list of bullet points, or say None if there are no items.
Ray buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?


In [57]:
print(proper_noun_query)

Given the following question, list the proper nouns.  Return the answer as a list of bullet points, or say None if there are no items.
Ray buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?


In [151]:
print(male_query)

Given the following question, list the males mentioned in the question.  Return the answer as a list of bullet points, or say None if there are no items.
Ray buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?


In [59]:
print(female_query)

Given the following question, list the females mentioned in the question.  Return the answer as a list of bullet points, or say None if there are no items
Ray buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?


In [60]:
util.call_gpt(noun_query)

<OpenAIObject chat.completion id=chatcmpl-8Fsrio2aYjJv6Rz195e70UA80wsAa at 0x144127e00> JSON: {
  "id": "chatcmpl-8Fsrio2aYjJv6Rz195e70UA80wsAa",
  "object": "chat.completion",
  "created": 1698796514,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "- Ray\n- pack\n- hamburger meat\n- box\n- crackers\n- bags\n- frozen vegetables\n- pack\n- cheese\n- grocery store\n- store rewards member\n- purchase\n- total grocery bill"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 116,
    "completion_tokens": 45,
    "total_tokens": 161
  }
}

In [153]:
def extract_list_items(list_string):
    items = list_string.split('\n')
    items = [item.replace('- ', '') for item in items]
    items = list(filter(lambda x: x != 'None', items))
    return items

In [159]:
def get_entities(preamble, query):
    """
    Given a preamble, fetch a list of entities.
    """
    prompt = f'{preamble}\n{query}'
    response = util.call_gpt(prompt)
    list_string = response.choices[0]['message']['content']
    if list_string == 'None':
        return []
    return extract_list_items(list_string)

In [83]:
def get_proper_nouns(query):
    return get_entities(proper_noun_preamble, query)

In [108]:
def get_nouns(query, no_numbers=True):
    nouns = get_entities(noun_preamble, query)
    if no_numbers:
        nouns = list(filter(lambda n: not re.match(r'.*\d.*', n), nouns))
    return nouns

In [91]:
def get_males(query):
    return get_entities(male_name_preamble, query)

In [92]:
def get_females(query):
    return get_entities(female_name_preamble, query)

In [84]:
get_proper_nouns(query)

['Ray',
 'hamburger meat',
 'crackers',
 'bags of frozen vegetables',
 'cheese',
 'grocery store']

In [109]:
nouns = get_nouns(query)
nouns

['Ray',
 'pack',
 'hamburger meat',
 'box',
 'crackers',
 'bags',
 'frozen vegetables',
 'pack',
 'cheese',
 'grocery store',
 'store rewards member',
 'purchase',
 'total grocery bill']

In [111]:
def get_non_name_nouns(names, nouns):
    noun_set = set(nouns)
    name_set = set(names)
    non_name_noun_set = set(noun_set).difference(name_set)
    return list(non_name_noun_set)

In [112]:
males = get_males(query)
males

['Ray']

In [115]:
females = get_females(query)
females

[]

In [116]:
people = males + females
people

['Ray']

In [117]:
nnn = get_non_name_nouns(people, nouns)
nnn

['pack',
 'crackers',
 'purchase',
 'box',
 'cheese',
 'hamburger meat',
 'bags',
 'grocery store',
 'frozen vegetables',
 'total grocery bill',
 'store rewards member']

In [23]:
weird_things = [
    'fluffernackle',
    'spiggotwhap',
    'blibberfudge',
    'quizzletoe',
    'plumbuzzle',
    'trinkleshuff',
    'flibberjig',
    'whackledorf',
    'snooptoggle',
    'zibberflap',
    'gobblequark',
    'dinglefrap',
    'snickerblast',
    'floopernoodle',
    'wobblegruff',
    'crinklethorp',
    'blubberclank',
    'mumblestitch',
    'quarkleflop',
    'slippleshank'
]

In [97]:
len(weird_things)

20

In [24]:
female_names = [
    'Isolde',
    'Seraphine',
    'Elowen',
    'Thalassa',
    'Vespera',
    'Calista',
    'Ondine',
    'Belphoebe',
    'Isabeau',
    'Eulalia',
]

In [25]:
male_names = [
    'Thorne',
    'Jorvik',
    'Elsdon',
    'Calix',
    'Vero',
    'Orlan',
    'Brevin',
    'Tyrus',
    'Halvard',
    'Dainan'
]

In [121]:
random.shuffle(male_names)

In [120]:
male_names

['Elsdon',
 'Brevin',
 'Calix',
 'Dainan',
 'Thorne',
 'Tyrus',
 'Jorvik',
 'Halvard',
 'Orlan',
 'Vero']

In [122]:
males

['Ray']

In [124]:
for real, fake in zip(males,male_names):
    print(f'{real}: {fake}')

Ray: Halvard


In [127]:
male_mapping = {real: fake for real, fake in zip(males, male_names)}

In [128]:
male_mapping

{'Ray': 'Halvard'}

In [129]:
query

'Ray buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?'

In [132]:
s = query
for real, fake in male_mapping.items():
    s = s.replace(real, fake)

In [133]:
s

'Halvard buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?'

In [134]:
random.shuffle(weird_things)

In [135]:
thing_mapping = {real: fake for real, fake in zip(nnn, weird_things)}

In [136]:
thing_mapping

{'pack': 'plumbuzzle',
 'crackers': 'wobblegruff',
 'purchase': 'zibberflap',
 'box': 'dinglefrap',
 'cheese': 'trinkleshuff',
 'hamburger meat': 'spiggotwhap',
 'bags': 'crinklethorp',
 'grocery store': 'floopernoodle',
 'frozen vegetables': 'blibberfudge',
 'total grocery bill': 'mumblestitch',
 'store rewards member': 'snickerblast'}

In [137]:
for real, fake in thing_mapping.items():
    s = s.replace(real, fake)

In [138]:
s

'Halvard buys a plumbuzzle of spiggotwhap for $5.00, a dinglefrap of wobblegruff for $3.50, 4 crinklethorp of blibberfudge at $2.00 per bag and a plumbuzzle of trinkleshuff for $3.50 at the floopernoodle.  Because he is a snickerblast, he gets 10% off of his zibberflap.  What does his mumblestitch come to?'

In [139]:
query

'Ray buys a pack of hamburger meat for $5.00, a box of crackers for $3.50, 4 bags of frozen vegetables at $2.00 per bag and a pack of cheese for $3.50 at the grocery store.  Because he is a store rewards member, he gets 10% off of his purchase.  What does his total grocery bill come to?'

In [158]:
def obfuscate_query(query):
    males = get_males(query)
    females = get_females(query)
    people = male_names + female_names
    nouns = get_nouns(query)
    non_name_nouns = get_non_name_nouns(people, nouns)
    random.shuffle(weird_things)
    random.shuffle(male_names)
    random.shuffle(female_names)
    
    male_mapping = {real: fake for real, fake in zip(males, male_names)}
    female_mapping = {real: fake for real, fake in zip(females, female_names)}
    thing_mapping = {real: fake for real, fake in zip(non_name_nouns, weird_things)}
    
    s = query
    for real, fake in male_mapping.items():
        s = s.replace(real, fake)
    for real, fake in female_mapping.items():
        s = s.replace(real, fake)
    for real, fake in thing_mapping.items():
        s = s.replace(real, fake)
        
    return s

In [160]:
obfuscate_query(query)

'crinklethorp buys a gobblequark of zibberflap for $5.00, a trinkleshuff of spiggotwhap for $3.50, 4 quizzletoes of quarkleflop at $2.00 per quizzletoe and a gobblequark of blibberfudge for $3.50 at the flibberjig.  Because he is a plumbuzzle, he gets 10% off of his snickerblast.  What does his mumblestitch come to?'